In [29]:
import numpy as np
import pandas as pd
# from icecream import ic
import subprocess
import json
import os
# import plotly.graph_objects as go
from matplotlib import pyplot as plt
# import geopandas as gpd
# from shapely.geometry import Polygon
# from shapely.ops import cascaded_union, unary_union
# from shapely import geometry
# from geopy import distance
%matplotlib inline
# import plotly.express as px 
from scipy import signal
from tqdm import tqdm
# from Bio import Phylo
# import ete3
import itertools
import sklearn

In [47]:
qwe = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/CRyPTIC_reuse_table_20231208.csv')

In [48]:
qwe['RIF_MIC'].value_counts()

RIF_MIC
<=0.06    2318
>4        2243
0.12      2106
>8        1771
0.06      1326
<=0.03     753
0.25       723
8.0        229
0.5        188
4.0        169
2.0        151
1.0        120
Name: count, dtype: int64

In [61]:
qwe[qwe['RIF_MIC'].isin(['>4','4.0', '2', '1', '0.5', '0.25', '0.12','<=0.06'])]['RIF_MIC'].value_counts()

RIF_MIC
<=0.06    2318
>4        2243
0.12      2106
0.25       723
0.5        188
4.0        169
Name: count, dtype: int64

In [ ]:
UKMYC5,RIF,8,>4,,,R
UKMYC5,RIF,7,4.0,1,9,R
UKMYC5,RIF,6,2.0,2,9,R
UKMYC5,RIF,5,1.0,3,9,R
UKMYC5,RIF,4,0.5,4,9,S
UKMYC5,RIF,3,0.25,5,9,S
UKMYC5,RIF,2,0.12,6,9,S
UKMYC5,RIF,1,<=0.06,7,9,S

In [ ]:
# UKMYC6,RIF,10,>8,,,R
# UKMYC6,RIF,9,8.0,2,3,R
UKMYC6,RIF,8,4.0,1,3,R
UKMYC6,RIF,7,2.0,1,2,R
UKMYC6,RIF,6,1.0,2,2,R
UKMYC6,RIF,5,0.5,3,2,S
UKMYC6,RIF,4,0.25,4,2,S
UKMYC6,RIF,3,0.12,5,2,S
# UKMYC6,RIF,2,0.06,6,2,S
# UKMYC6,RIF,1,<=0.03,7,2,S

In [49]:
qwe.columns

Index(['ENA_RUN', 'UNIQUEID', 'AMI_BINARY_PHENOTYPE', 'BDQ_BINARY_PHENOTYPE',
       'CFZ_BINARY_PHENOTYPE', 'DLM_BINARY_PHENOTYPE', 'EMB_BINARY_PHENOTYPE',
       'ETH_BINARY_PHENOTYPE', 'INH_BINARY_PHENOTYPE', 'KAN_BINARY_PHENOTYPE',
       'LEV_BINARY_PHENOTYPE', 'LZD_BINARY_PHENOTYPE', 'MXF_BINARY_PHENOTYPE',
       'RIF_BINARY_PHENOTYPE', 'RFB_BINARY_PHENOTYPE', 'AMI_MIC', 'BDQ_MIC',
       'CFZ_MIC', 'DLM_MIC', 'EMB_MIC', 'ETH_MIC', 'INH_MIC', 'KAN_MIC',
       'LEV_MIC', 'LZD_MIC', 'MXF_MIC', 'RIF_MIC', 'RFB_MIC',
       'AMI_PHENOTYPE_QUALITY', 'BDQ_PHENOTYPE_QUALITY',
       'CFZ_PHENOTYPE_QUALITY', 'DLM_PHENOTYPE_QUALITY',
       'EMB_PHENOTYPE_QUALITY', 'ETH_PHENOTYPE_QUALITY',
       'INH_PHENOTYPE_QUALITY', 'KAN_PHENOTYPE_QUALITY',
       'LEV_PHENOTYPE_QUALITY', 'LZD_PHENOTYPE_QUALITY',
       'MXF_PHENOTYPE_QUALITY', 'RIF_PHENOTYPE_QUALITY',
       'RFB_PHENOTYPE_QUALITY', 'ENA_SAMPLE', 'VCF', 'REGENOTYPED_VCF'],
      dtype='object')

In [30]:
def value_counts_list(lst):
    """
    Computes the frequency count of unique elements in a list and returns a dictionary, sorted by frequency count in
    descending order.

    Args:
    - lst (list): List of elements

    Returns:
    - dict: Dictionary with unique elements as keys and their frequency count as values, sorted by frequency count
    in descending order
    """
    value_counts = {}
    for item in lst:
        if item in value_counts:
            value_counts[item] += 1
        else:
            value_counts[item] = 1
    sorted_value_counts = dict(sorted(value_counts.items(), key=lambda x: x[1], reverse=True))
    return sorted_value_counts

def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [63]:
variants = pd.read_csv('variants_full.csv')


In [64]:
variants = variants[variants['type'] != 'synonymous_variant']

In [65]:
cryptic = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/CRyPTIC_reuse_table_20221019_cleaned.csv')

In [66]:
cryptic = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/CRyPTIC_reuse_table_20221019_cleaned.csv')
overlap = set(variants['sample_id']).intersection(set(cryptic['wgs_id'].to_list()))
variants = variants[variants['sample_id'].isin(overlap)]
variants['SNP'] = variants['gene'] + '-'+ variants['change']

In [68]:
len(variants['SNP'].unique())

21766

In [36]:
def compare_snp_lists_with_values(set_list, query_list, values_list):
    # Create a dictionary from query_list and values_list for faster lookups
    query_dict = dict(zip(query_list, values_list))
    
    # Initialize output list with zeros
    output_list = [0] * len(set_list)
    
    # Iterate through the set list and update the output list
    for i, snp in enumerate(set_list):
        if snp in query_dict:
            # Use the corresponding value from values_list instead of 1
            output_list[i] = query_dict[snp]
    
    return output_list

# Example usage:
set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
query_list = ['SNP2', 'SNP4']
values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
output_list = compare_snp_lists_with_values(set_list, query_list, values_list)
print(output_list)  # Expected output: [0, 5, 0, 10]


def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
    # Create a dictionary from query_list and values_list for direct mapping
    query_dict = dict(zip(query_list, values_list))
    
    # Use list comprehension to build the output list directly
    output_list = [query_dict.get(snp, 0) for snp in set_list]
    
    return output_list

# Example usage
set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
query_list = ['SNP2', 'SNP4']
values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data

[0, 5, 0, 10]
[0, 5, 0, 10]


# Running for all data

### Below is the main code for generating snp data, runs for a long time
trying loading already generated data: aa_array = np.loadtxt('data_aa/aa_data.csv', delimiter=',')   # X is an array

In [37]:
aa = []
all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table 
for x in tqdm(variants['sample_id'].unique()):
    aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
    # print('SNP')
    # print(variants[variants['sample_id']==x]['SNP'].to_list()), 
    # print('freq')
    # print(variants[variants['sample_id']==x]['freq'].to_list())
    # break

  0%|          | 3/11920 [00:00<08:42, 22.79it/s]

  0%|          | 36/11920 [00:01<08:23, 23.59it/s]


KeyboardInterrupt: 

In [25]:
aa_array = np.array(aa)
aa_array[aa_array < 0.8] = 0

In [80]:
np.savetxt('data_aa/aa_data_col.csv',variants['SNP'].unique(),  delimiter=',', fmt='%s')
np.savetxt('data_aa/aa_data_row.csv',variants['sample_id'].unique(),  delimiter=',', fmt='%s')

In [31]:
# loading already generated data: ~ 3 min to load
aa_array = np.loadtxt('data_aa/aa_data.csv', delimiter=',')   # X is an array

In [69]:
cryptic_mic = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data1/CRyPTIC_reuse_table_20221019_cleaned.csv')

In [50]:
cryptic_mic_ori = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/CRyPTIC_reuse_table_20221019.csv')

In [70]:
mic_aa = cryptic_mic[cryptic_mic['wgs_id'].isin(variants['sample_id'].unique())]#.iloc[:,14:27]

getting mic into the right order in terms of sample

In [71]:
#ordering mic like in aa_array - make sure the samples are in the correct order
mic_aa['wgs_id'] = pd.Categorical(mic_aa['wgs_id'], categories=variants['sample_id'].unique().tolist(), ordered=True)
mic_aa = mic_aa.sort_values('wgs_id')

/tmp/ipykernel_119769/4217528871.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa['wgs_id'] = pd.Categorical(mic_aa['wgs_id'], categories=variants['sample_id'].unique().tolist(), ordered=True)


In [ ]:
variants['sample_id'].unique().tolist() == mic_aa['wgs_id'].unique().tolist()

In [76]:
aa_array.shape

(11920, 21766)

In [28]:
np.savetxt('data_aa/aa_data_train.csv', aa_array[:int(aa_array.shape[0]*0.9), :], delimiter=',', fmt='%s')
# Save the remaining 10% of rows to another CSV file
np.savetxt('data_aa/aa_data_test.csv', aa_array[int(aa_array.shape[0]*0.9):, :], delimiter=',', fmt='%s')

In [56]:
mic_aa = mic_aa.iloc[:,14:27].astype('float')
mic_aa.to_csv('data_aa/mic_aa.csv', index=False)

In [57]:
mic_aa.iloc[:int(mic_aa.shape[0]*0.9),:].to_csv('data_aa/mic_aa_train.csv', index=False)
mic_aa.iloc[int(mic_aa.shape[0]*0.9):,:].to_csv('data_aa/mic_aa_test.csv', index=False)

In [58]:
mic_aa.shape

(11920, 13)

In [59]:
aa_array.shape

(11920, 27063)

In [59]:
mic_aa.iloc[:,42]

8157     ERR4829684
11361    ERR8975196
1244     ERR3287611
7641     ERR4828904
9905     ERR4831761
            ...    
5125     ERR4812175
2539     ERR4797355
4936     ERR4811968
464      ERR2510245
2252     ERR4797048
Name: wgs_id, Length: 11920, dtype: category
Categories (11920, object): ['ERR4829684' < 'ERR8975196' < 'ERR3287611' < 'ERR4828904' ... 'ERR4797355' < 'ERR4811968' < 'ERR2510245' < 'ERR4797048']

# Binning MIC

### Overall

In [97]:
# mic_aa = mic_aa.iloc[:,14:27].astype('float')
mic_aa = pd.read_csv('data_aa/mic_aa.csv')

In [52]:
my_dict = {0.5: 24570,
 0.03: 19597,
 2.0: 13160,
 0.25: 12768,
 1.0: 12519,
 0.12: 9215,
 4.0: 8419,
 0.06: 8391,
 0.125: 7181,
 0.0075: 6099,
 8.0: 5915,
 0.015: 4858,
 16.0: 3971,
 0.05: 3388,
 0.004: 3154,
 3.2: 2468,
 0.01125: 2433,
 1.6: 1368,
 32.0: 1034,
 0.2: 355,
 0.8: 341,
 6.4: 335,
 0.4: 304,
 0.1: 271,
 25.6: 241,
 12.8: 198,
 0.15: 72,
 64.0: 14}
sorted_dict = dict(sorted(my_dict.items()))
for x, y in sorted_dict.items():
    print(x,'-', y)

0.004 - 3154
0.0075 - 6099
0.01125 - 2433
0.015 - 4858
0.03 - 19597
0.05 - 3388
0.06 - 8391
0.1 - 271
0.12 - 9215
0.125 - 7181
0.15 - 72
0.2 - 355
0.25 - 12768
0.4 - 304
0.5 - 24570
0.8 - 341
1.0 - 12519
1.6 - 1368
2.0 - 13160
3.2 - 2468
4.0 - 8419
6.4 - 335
8.0 - 5915
12.8 - 198
16.0 - 3971
25.6 - 241
32.0 - 1034
64.0 - 14



=== mic < 0.01
0.004 - 3154
0.0075 - 6099

=== 0.01 <= mic < 0.02 
0.01125 - 2433
0.015 - 4858

=== 0.02 <= mic < 0.02 
0.03 - 19597

=== 0.05 <= mic < 0.1 
0.05 - 3388
0.06 - 8391

=== 0.1 <= mic < 0.15 
0.1 - 271
0.12 - 9215
0.125 - 7181

=== 0.15 <= mic < 0.4 
0.15 - 72
0.2 - 355
0.25 - 12768

=== 0.5 <= mic < 0.8
0.4 - 304
0.5 - 24570

=== 0.8 <= mic < 1
0.8 - 341
1.0 - 12519

=== 1 <= mic < 3.2
1.6 - 1368
2.0 - 13160

=== 3.2 <= mic < 8
3.2 - 2468
4.0 - 8419
6.4 - 335

=== mic >= 8
8.0 - 5915
12.8 - 198
16.0 - 3971
25.6 - 241
32.0 - 1034
64.0 - 14

In [63]:

# === mic < 0.01
# 0.004 - 3154
# 0.0075 - 6099
print(sum([3154,6099])) 

# === 0.01 <= mic < 0.02 
# 0.01125 - 2433
# 0.015 - 4858
print(sum([2433, 4858]))

# === 0.02 <= mic < 0.02 
# 0.03 - 19597
print(sum([19597]))

# === 0.05 <= mic < 0.1 
# 0.05 - 3388
# 0.06 - 8391
print(sum([3388, 8391]))

# === 0.1 <= mic < 0.15 
# 0.1 - 271
# 0.12 - 9215
# 0.125 - 7181
print(sum([271,9215,7181]))

# === 0.15 <= mic < 0.4 
# 0.15 - 72
# 0.2 - 355
# 0.25 - 12768
print(sum([72,355,12768]))

# === 0.5 <= mic < 0.8
# 0.4 - 304
# 0.5 - 24570
print(sum([304,24570]))

# === 0.8 <= mic < 1
# 0.8 - 341
# 1.0 - 12519
print(sum([341,12519]))

# === 1 <= mic < 3.2
# 1.6 - 1368
# 2.0 - 13160
print(sum([1368,13160]))

# === 3.2 <= mic < 8
# 3.2 - 2468
# 4.0 - 8419
# 6.4 - 335
print(sum([2468,8419,335]))

# === mic >= 8
# 8.0 - 5915
# 12.8 - 198
# 16.0 - 3971
# 25.6 - 241
# 32.0 - 1034
# 64.0 - 14
print(sum([5915,198,3971,241,1034,14]))


9253
7291
19597
11779
16667
13195
24874
12860
14528
11222
11373


In [62]:
print(sum([3154,6099])) 
print(sum([2433, 4858]))
print(sum([19597]))
print(sum([3388, 8391]))
print(sum([271,9215,7181]))
print(sum([72,355,12768]))
print(sum([304,24570]))
print(sum([341,12519]))
print(sum([1368,13160]))
print(sum([2468,8419,335]))
print(sum([5915,198,3971,241,1034,14]))


9253
7291
19597
11779
16667
13195
24874
12860
14528
11222
11373


## RIF

In [20]:
mic_aa = pd.read_csv('data_aa/mic_aa.csv')

In [85]:
rif_ori_sorted_dict = dict(sorted(dict(cryptic_mic_ori['RIF_MIC'].value_counts()).items()))

In [74]:
# Convert list values according to provided ranges into numbers corresponding to which range the value falls in

# Define the ranges and their corresponding labels
ranges = {
    (0.015, 0.05): 1,  # Includes 0.015 and 0.03
    (0.05, 0.15): 2,   # Includes 0.06
    (0.12, 0.25): 3,   # Includes 0.12
    (0.25, 1.0): 4,    # Includes 0.25 and 0.5
    (1.0, 8.0): 5,     # Includes 1.0, 2.0, and 4.0
    (8.0, float('inf')): 6  # Includes 8.0 and 16.0
}

# Sample MIC values
mic_values = [0.015, 0.03, 0.06, 0.12, 0.25, 0.5, 1.0, 2.0, 4.0, 8.0, 16.0]

# Function to convert MIC values to corresponding range numbers
def convert_mic_to_range(mic_values, ranges):
    converted_values = []
    for mic in mic_values:
        for r in ranges:
            if r[0] <= mic <= r[1]:
                converted_values.append(ranges[r])
                break
    return converted_values

# Convert the MIC values
converted_values = convert_mic_to_range(mic_values, ranges)
converted_values


[1, 1, 2, 2, 3, 4, 4, 5, 5, 5, 6]

In [75]:
converted_values = convert_mic_to_range(, ranges)


[1, 1, 2, 2, 3, 4, 4, 5, 5, 5, 6]

In [67]:
dict(mic_aa['RIF_MIC'].value_counts())

{8.0: 2374,
 0.03: 2262,
 0.12: 2082,
 16.0: 1631,
 0.06: 1316,
 0.015: 751,
 0.25: 715,
 0.5: 183,
 4.0: 160,
 2.0: 148,
 1.0: 114}

In [68]:
rif_sorted_dict = dict(sorted(dict(mic_aa['RIF_MIC'].value_counts()).items()))


In [115]:
# rif_sorted_dict # 0.5 is the subsecptibility threshold
# === mic < 0.05
# 0.015: 751,
# 0.03: 2262,
print(sum([751,2262]))

# === <=0.05 mic < 0.12
# 0.06: 1316,
print(sum([1316]))

# === 0.12<= mic  < 0.25
# 0.12: 2082,
print(sum([2082]))

# === 0.25 <= mic <=0.5
# 0.25: 715,
# 0.5: 183,
print(sum([715,183]))

# === 0.5 < mic < 8
# 1.0: 114,
# 2.0: 148,
# 4.0: 160,
print(sum([114,148,160]))

# === mic >= 8
# 8.0: 2374,
# 16.0: 1631
print(sum([2374,1631]))

3013
1316
2082
898
422
4005


In [21]:

# Encoding MIC values based on specified ranges
# encoded_mic = []
# for value in mic_aa['RIF_MIC']:
#     if value < 0.05:
#         encoded_mic.append(1)  # mic < 0.05
#     elif 0.05 <= value < 0.12:
#         encoded_mic.append(2)  # 0.05 <= mic < 0.12
#     elif 0.12 <= value < 0.25:
#         encoded_mic.append(3)  # 0.12 <= mic < 0.25
#     elif 0.25 <= value <= 0.5:
#         encoded_mic.append(4)  # 0.25 <= mic <= 0.5
#     elif 0.5 < value < 8:
#         encoded_mic.append(5)  # 0.5 < mic < 8
#     elif value >= 8:
#         encoded_mic.append(6)  # mic >= 8
#     else:
#         # print(value)
#         encoded_mic.append(np.nan)


encoded_mic = []
for value in mic_aa['RIF_MIC']:
    if value <= 0.2:
        encoded_mic.append(1)  # mic < 0.05
    elif 0.2 <= value < 0.5:
        encoded_mic.append(2)  # 0.05 <= mic < 0.12
    elif 0.5 <= value < 8:
        encoded_mic.append(3)  # 0.12 <= mic < 0.25
    elif value >= 8:
        encoded_mic.append(4)  # mic >= 8
    else:
        # print(value)
        encoded_mic.append(np.nan)

    
        
# encoded_mic = []
# for value in mic_aa['RIF_MIC']:
#     if value < 0.05:
#         encoded_mic.append(1)  # mic < 0.05
#     elif 0.05 <= value < 0.12:
#         encoded_mic.append(2)  # 0.05 <= mic < 0.12
#     elif 0.12 <= value < 0.25:
#         encoded_mic.append(3)  # 0.12 <= mic < 0.25
#     elif 0.25 <= value < 8:
#         encoded_mic.append(4)  # 0.25 <= mic <= 0.5
#     elif value >= 8:
#         encoded_mic.append(5)  # mic >= 8
#     else:
#         # print(value)
#         encoded_mic.append(np.nan)

mic_aa['RIF_MIC'] = encoded_mic

In [22]:
mic_aa['RIF_MIC'].value_counts()

RIF_MIC
1.0    6411
4.0    4005
2.0     715
3.0     605
Name: count, dtype: int64

In [23]:
mic_aa.iloc[:int(mic_aa.shape[0]*0.9),:].to_csv('data_aa/mic_aa_train_rif.csv', index=False)
mic_aa.iloc[int(mic_aa.shape[0]*0.9):,:].to_csv('data_aa/mic_aa_test_rif.csv', index=False)

# Stratified - overall

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit


In [ ]:
# Convert the labels DataFrame to a NumPy array if it's for multilabel stratification
labels = mic_aa.to_numpy()
msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)

# Since MultilabelStratifiedShuffleSplit works with indices, we generate an index array
indices = np.arange(features.shape[0])

# Perform the split
for train_index, test_index in msss.split(indices, labels):
    train_indices, test_indices = train_index, test_index

# Using indices to create feature subsets
train_features = features[train_indices]
test_features = features[test_indices]

# Using indices to create label subsets (converting them back to DataFrame if needed)
train_labels = pd.DataFrame(labels[train_indices], columns=labels_df.columns)
test_labels = pd.DataFrame(labels[test_indices], columns=labels_df.columns)



# Getting rpob aa snp data

In [ ]:
all_snp

In [84]:
variants_ind = variants[variants['gene']=='rpoB']
aa_rpoB = []
all_snp = variants_ind['SNP'].unique()

for x in tqdm(variants['sample_id'].unique()):
    if x in variants_ind['sample_id'].unique():
        aa_rpoB.append(compare_snp_lists_with_values(all_snp, variants_ind[variants_ind['sample_id']==x]['SNP'].to_list(), variants_ind[variants_ind['sample_id']==x]['freq'].to_list()))
    else:
        aa_rpoB.append([0]*len(all_snp))
aa_rpoB_array = np.array(aa_rpoB)
np.savetxt('data_aa/aa_rpoB.csv', aa_rpoB_array, delimiter=',')   # X is an array

100%|██████████| 11920/11920 [00:31<00:00, 373.60it/s]


In [131]:
np.savetxt('data_aa/train_aa_rpoB.csv', aa_rpoB_array[:int(0.9*aa_rpoB_array.shape[0])], delimiter=',')   # X 
np.savetxt('data_aa/test_aa_rpoB.csv', aa_rpoB_array[int(0.9*aa_rpoB_array.shape[0]):], delimiter=',')   # X 

In [ ]:
np.savetxt('data_aa/aa_rpoB.csv', aa_rpoB_array, delimiter=',')   # X is an array

# Getting eth aa snp data

In [87]:
variants_ind = variants[variants['gene'].isin(['embA','embB','embC'])]
aa_emb = []
all_snp = variants_ind['SNP'].unique()

for x in tqdm(variants['sample_id'].unique()):
    if x in variants_ind['sample_id'].unique():
        aa_emb.append(compare_snp_lists_with_values(all_snp, variants_ind[variants_ind['sample_id']==x]['SNP'].to_list(), variants_ind[variants_ind['sample_id']==x]['freq'].to_list()))
    else:
        aa_emb.append([0]*len(all_snp))
aa_emb_array = np.array(aa_emb)
np.savetxt('data_aa/aa_emb.csv', aa_emb_array, delimiter=',')   # X is an array

100%|██████████| 11920/11920 [02:10<00:00, 91.03it/s]


In [132]:
np.savetxt('data_aa/aa_emb.csv', aa_emb_array, delimiter=',')   # X is an array
np.savetxt('data_aa/train_aa_emb.csv', aa_emb_array[:int(0.9*aa_emb_array.shape[0])], delimiter=',')   # X 
np.savetxt('data_aa/test_aa_emb.csv', aa_emb_array[int(0.9*aa_emb_array.shape[0]):], delimiter=',')   # X 

# Getting corresponding mic snp dataset for the above

In [113]:
cryptic_mic['wgs_id'] = pd.Categorical(cryptic_mic['wgs_id'], categories=variants['sample_id'].unique().tolist(), ordered=True)
# Sort the DataFrame by the 'wgs_id' column

cryptic_mic_ordered = cryptic_mic.sort_values('wgs_id')
overlap = set(variants['sample_id']).intersection(set(cryptic['wgs_id'].to_list()))
cryptic_mic_ordered = cryptic_mic_ordered[cryptic_mic_ordered['wgs_id'].isin(overlap)]

In [114]:
np.savetxt('data_aa/ETH_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['ETH_MIC']]), delimiter=',')   # X is an array
np.savetxt('data_aa/train_ETH_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['ETH_MIC']])[:int(0.9*cryptic_mic_ordered.shape[0])], delimiter=',')   # X is an array
np.savetxt('data_aa/test_ETH_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['ETH_MIC']])[int(0.9*cryptic_mic_ordered.shape[0]):], delimiter=',')   # X is an array

np.savetxt('data_aa/RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']]), delimiter=',')   # X is an array
np.savetxt('data_aa/train_RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']])[:int(0.9*cryptic_mic_ordered.shape[0])], delimiter=',')   # X is an array
np.savetxt('data_aa/test_RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']])[int(0.9*cryptic_mic_ordered.shape[0]):], delimiter=',')   # X is an array

In [92]:
cryptic_mic['ETH_MIC']

Index(['UNIQUEID', 'AMI_BINARY_PHENOTYPE', 'BDQ_BINARY_PHENOTYPE',
       'CFZ_BINARY_PHENOTYPE', 'DLM_BINARY_PHENOTYPE', 'EMB_BINARY_PHENOTYPE',
       'ETH_BINARY_PHENOTYPE', 'INH_BINARY_PHENOTYPE', 'KAN_BINARY_PHENOTYPE',
       'LEV_BINARY_PHENOTYPE', 'LZD_BINARY_PHENOTYPE', 'MXF_BINARY_PHENOTYPE',
       'RIF_BINARY_PHENOTYPE', 'RFB_BINARY_PHENOTYPE', 'AMI_MIC', 'BDQ_MIC',
       'CFZ_MIC', 'DLM_MIC', 'EMB_MIC', 'ETH_MIC', 'INH_MIC', 'KAN_MIC',
       'LEV_MIC', 'LZD_MIC', 'MXF_MIC', 'RIF_MIC', 'RFB_MIC',
       'AMI_PHENOTYPE_QUALITY', 'BDQ_PHENOTYPE_QUALITY',
       'CFZ_PHENOTYPE_QUALITY', 'EMB_PHENOTYPE_QUALITY',
       'ETH_PHENOTYPE_QUALITY', 'INH_PHENOTYPE_QUALITY',
       'KAN_PHENOTYPE_QUALITY', 'LEV_PHENOTYPE_QUALITY',
       'LZD_PHENOTYPE_QUALITY', 'MXF_PHENOTYPE_QUALITY',
       'RIF_PHENOTYPE_QUALITY', 'RFB_PHENOTYPE_QUALITY', 'ENA_SAMPLE', 'VCF',
       'REGENOTYPED_VCF', 'wgs_id'],
      dtype='object')

In [ ]:
from tqdm import tqdm
import numpy as np

def convert_to_coordinate_value(input_list):
    return [(i, val) for i, val in enumerate(input_list) if val != 0]

# Generate coordinate values
aa_coord = [convert_to_coordinate_value(x) for x in tqdm(aa_data_freq)]

# Find the length of the longest list
max_length = max(len(item) for item in aa_coord)

# Pad the lists to have uniform length
padded_aa_coord = [item + [(0, 0)] * (max_length - len(item)) for item in aa_coord]

# Convert to a high-dimensional NumPy array
aa_coord_array = np.array(padded_aa_coord)

# The result is a high-dimensional NumPy array
print(aa_coord_array.shape)


In [ ]:
from tqdm import tqdm
import numpy as np

def convert_to_coordinate_value(input_list):
    return [(i, val) for i, val in enumerate(input_list) if val != 0]

# Generate coordinate values
aa_coord = [convert_to_coordinate_value(x) for x in tqdm(aa_data_freq)]

# Find the length of the longest list
max_length = max(len(item) for item in aa_coord)

# Pad the lists to have uniform length
padded_aa_coord = [item + [(0, 0)] * (max_length - len(item)) for item in aa_coord]

# Convert to a high-dimensional NumPy array
aa_coord_array = np.array(padded_aa_coord)

# The result is a high-dimensional NumPy array
print(aa_coord_array.shape)


In [ ]:
for x in np.unique(variants['sample_id']):
     print(variants[variants['sample_id'] == 'ERR9787047'].shape)

In [7]:
variants['change'].value_counts()

n.-187C>T               135403
p.Glu21Gln              133939
c.33A>G                 133602
c.-165T>C               133108
p.Ile948Val             133064
                         ...  
c.-4164_*1837del             1
p.Ala876Ser                  1
c.2192_2193insGCCTAG         1
c.536_537insGCCCG            1
n.1011T>G                    1
Name: change, Length: 109920, dtype: int64

# Classification targets

In [82]:
cryptic_clean = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/CRyPTIC_reuse_table_20221019_cleaned.csv')

In [79]:
rif_target = pd.read_csv('data_aa/RIF_MIC.csv',header=None)

In [84]:
cryptic_clean['RIF_MIC']

0         0.12
1         0.12
2         0.03
3         0.12
4         0.03
         ...  
12280    16.00
12281    16.00
12282    16.00
12283     0.06
12284    16.00
Name: RIF_MIC, Length: 12285, dtype: float64

In [19]:
print(np.sort(np.unique(rif_target.values)))

[1.5e-02 3.0e-02 6.0e-02 1.2e-01 2.5e-01 5.0e-01 1.0e+00 2.0e+00 4.0e+00
 8.0e+00 1.6e+01     nan]


In [22]:
cryptic_mic_ordered.columns

Index(['UNIQUEID', 'AMI_BINARY_PHENOTYPE', 'BDQ_BINARY_PHENOTYPE',
       'CFZ_BINARY_PHENOTYPE', 'DLM_BINARY_PHENOTYPE', 'EMB_BINARY_PHENOTYPE',
       'ETH_BINARY_PHENOTYPE', 'INH_BINARY_PHENOTYPE', 'KAN_BINARY_PHENOTYPE',
       'LEV_BINARY_PHENOTYPE', 'LZD_BINARY_PHENOTYPE', 'MXF_BINARY_PHENOTYPE',
       'RIF_BINARY_PHENOTYPE', 'RFB_BINARY_PHENOTYPE', 'AMI_MIC', 'BDQ_MIC',
       'CFZ_MIC', 'DLM_MIC', 'EMB_MIC', 'ETH_MIC', 'INH_MIC', 'KAN_MIC',
       'LEV_MIC', 'LZD_MIC', 'MXF_MIC', 'RIF_MIC', 'RFB_MIC',
       'AMI_PHENOTYPE_QUALITY', 'BDQ_PHENOTYPE_QUALITY',
       'CFZ_PHENOTYPE_QUALITY', 'EMB_PHENOTYPE_QUALITY',
       'ETH_PHENOTYPE_QUALITY', 'INH_PHENOTYPE_QUALITY',
       'KAN_PHENOTYPE_QUALITY', 'LEV_PHENOTYPE_QUALITY',
       'LZD_PHENOTYPE_QUALITY', 'MXF_PHENOTYPE_QUALITY',
       'RIF_PHENOTYPE_QUALITY', 'RFB_PHENOTYPE_QUALITY', 'ENA_SAMPLE', 'VCF',
       'REGENOTYPED_VCF', 'wgs_id'],
      dtype='object')

In [25]:
for x in ['AMI_MIC', 'BDQ_MIC','CFZ_MIC', 'DLM_MIC', 'EMB_MIC', 'ETH_MIC', 'INH_MIC', 'KAN_MIC','LEV_MIC', 'LZD_MIC', 'MXF_MIC', 'RIF_MIC', 'RFB_MIC']:
    print(x)
    print(np.sort(np.unique(cryptic_mic_ordered[x].values)))

AMI_MIC
[ 0.125  0.5    1.     2.     4.     8.    16.    32.       nan]
BDQ_MIC
[0.004  0.0075 0.015  0.03   0.06   0.12   0.25   0.5    1.     2.
 4.        nan]
CFZ_MIC
[0.015 0.03  0.06  0.12  0.25  0.5   1.    2.    4.    8.      nan]
DLM_MIC
[0.004  0.0075 0.015  0.03   0.06   0.12   0.25   0.5    1.     2.
    nan]
EMB_MIC
[3.00e-02 1.20e-01 1.25e-01 2.50e-01 5.00e-01 1.00e+00 2.00e+00 4.00e+00
 8.00e+00 1.60e+01 3.20e+01 6.40e+01      nan]
ETH_MIC
[ 0.125  0.5    1.     2.     4.     8.    16.       nan]
INH_MIC
[1.125e-02 5.000e-02 1.000e-01 2.000e-01 4.000e-01 8.000e-01 1.600e+00
 3.200e+00 6.400e+00 1.280e+01 2.560e+01       nan]
KAN_MIC
[ 0.5  2.   4.   8.  16.  32.   nan]
LEV_MIC
[ 0.06  0.25  0.5   1.    2.    4.    8.   16.     nan]
LZD_MIC
[0.03 0.06 0.12 0.15 0.25 0.5  1.   2.   4.   8.    nan]
MXF_MIC
[0.03 0.12 0.25 0.5  1.   2.   4.   8.    nan]
RIF_MIC
[1.5e-02 3.0e-02 6.0e-02 1.2e-01 2.5e-01 5.0e-01 1.0e+00 2.0e+00 4.0e+00
 8.0e+00 1.6e+01     nan]
RFB_MIC
[0.03 0

In [90]:
import pandas as pd
import numpy as np

# Sample data
data = {
    'Values': [1.5e-02, 3.0e-02, 6.0e-02, 1.2e-01, 2.5e-01, 5.0e-01, 1.0e+00, 2.0e+00, 4.0e+00, 8.0e+00, 1.6e+01, np.nan]
}

def one_hot_encode_df(data):
    # Create a DataFrame from the data
    df = pd.DataFrame(data)
    
    # One-hot encode the data, converting NaN values as well
    # 'prefix' can be adjusted based on preference for column naming
    one_hot_encoded_df = pd.get_dummies(df.iloc[:,0], dummy_na=True)
    
    return one_hot_encoded_df

df = one_hot_encode_df(data)
print(df)


    0.015   0.030   0.060   0.120   0.250   0.500   1.000   2.000   4.000   \
0        1       0       0       0       0       0       0       0       0   
1        0       1       0       0       0       0       0       0       0   
2        0       0       1       0       0       0       0       0       0   
3        0       0       0       1       0       0       0       0       0   
4        0       0       0       0       1       0       0       0       0   
5        0       0       0       0       0       1       0       0       0   
6        0       0       0       0       0       0       1       0       0   
7        0       0       0       0       0       0       0       1       0   
8        0       0       0       0       0       0       0       0       1   
9        0       0       0       0       0       0       0       0       0   
10       0       0       0       0       0       0       0       0       0   
11       0       0       0       0       0       0       0      

In [91]:
cryptic_clean['RIF_MIC'].unique()

array([1.2e-01, 3.0e-02, 8.0e+00, 2.0e+00,     nan, 2.5e-01, 1.0e+00,
       5.0e-01, 4.0e+00, 1.6e+01, 1.5e-02, 6.0e-02])

In [110]:
cryptic_clean

,UNIQUEID,AMI_BINARY_PHENOTYPE,BDQ_BINARY_PHENOTYPE,CFZ_BINARY_PHENOTYPE,DLM_BINARY_PHENOTYPE,EMB_BINARY_PHENOTYPE,ETH_BINARY_PHENOTYPE,INH_BINARY_PHENOTYPE,KAN_BINARY_PHENOTYPE,LEV_BINARY_PHENOTYPE,...,KAN_PHENOTYPE_QUALITY,LEV_PHENOTYPE_QUALITY,LZD_PHENOTYPE_QUALITY,MXF_PHENOTYPE_QUALITY,RIF_PHENOTYPE_QUALITY,RFB_PHENOTYPE_QUALITY,ENA_SAMPLE,VCF,REGENOTYPED_VCF,wgs_id
0,site.05.subj.PTAN-0001.lab.TAN-001.iso.1,S,S,S,S,S,S,S,S,S,...,HIGH,LOW,HIGH,HIGH,HIGH,HIGH,ERS1987021,00/00/04/90/490/site.05.iso.1.subject.PTAN-000...,00/00/04/90/490/site.05.iso.1.subject.PTAN-000...,ERR2184202
1,site.05.subj.PTAN-0002.lab.TAN-004.iso.1,S,S,S,S,S,S,S,S,S,...,HIGH,HIGH,LOW,LOW,LOW,HIGH,ERS1987022,00/00/04/91/491/site.05.iso.1.subject.PTAN-000...,00/00/04/91/491/site.05.iso.1.subject.PTAN-000...,ERR2184203
2,site.05.subj.PTAN-0003.lab.TAN-006.iso.1,S,S,S,S,S,S,S,S,S,...,HIGH,HIGH,HIGH,LOW,HIGH,HIGH,ERS1987023,00/00/04/92/492/site.05.iso.1.subject.PTAN-000...,00/00/04/92/492/site.05.iso.1.subject.PTAN-000...,ERR2184204
3,site.05.subj.PTAN-0004.lab.TAN-007.iso.1,S,S,S,S,S,NaN,S,S,S,...,HIGH,HIGH,LOW,HIGH,HIGH,HIGH,ERS1987024,00/00/04/93/493/site.05.iso.1.subject.PTAN-000...,00/00/04/93/493/site.05.iso.1.subject.PTAN-000...,ERR2184205
4,site.05.subj.PTAN-0005.lab.TAN-010.iso.1,S,S,S,S,S,S,S,S,S,...,HIGH,HIGH,LOW,HIGH,HIGH,HIGH,ERS1987025,00/00/04/94/494/site.05.iso.1.subject.PTAN-000...,00/00/04/94/494/site.05.iso.1.subject.PTAN-000...,ERR2184206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12280,site.11.subj.XTB_18-071.lab.XTB_18-071.iso.1,S,S,S,S,R,R,R,S,R,...,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,ERS6421155,00/01/47/01/14701/site.11.iso.1.subject.XTB_18...,00/01/47/01/14701/site.11.iso.1.subject.XTB_18...,SAMEA8741572
12281,site.11.subj.XTB_18-072.lab.XTB_18-072.iso.1,S,S,S,S,I,I,R,S,R,...,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,ERS6421156,00/01/47/02/14702/site.11.iso.1.subject.XTB_18...,00/01/47/02/14702/site.11.iso.1.subject.XTB_18...,SAMEA8741573
12282,site.11.subj.XTB_18-074.lab.XTB_18-074.iso.1,S,S,S,S,I,R,R,R,S,...,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,ERS6421157,00/01/47/03/14703/site.11.iso.1.subject.XTB_18...,00/01/47/03/14703/site.11.iso.1.subject.XTB_18...,SAMEA8741574
12283,site.11.subj.XTB_18-075.lab.XTB_18-075.iso.1,S,S,S,S,S,R,R,S,S,...,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,ERS6421158,00/01/47/04/14704/site.11.iso.1.subject.XTB_18...,00/01/47/04/14704/site.11.iso.1.subject.XTB_18...,SAMEA8741575


In [94]:
df.columns

Float64Index([0.015, 0.03, 0.06, 0.12, 0.25, 0.5, 1.0, 2.0, 4.0, 8.0, 16.0,
              nan],
             dtype='float64')

In [104]:
df = one_hot_encode_df(cryptic_clean['RIF_MIC'])
df.drop(columns=np.nan, inplace=True)

In [105]:
df

,0.015,0.030,0.060,0.120,0.250,0.500,1.000,2.000,4.000,8.000,16.000
0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
12280,0,0,0,0,0,0,0,0,0,0,1
12281,0,0,0,0,0,0,0,0,0,0,1
12282,0,0,0,0,0,0,0,0,0,0,1
12283,0,0,1,0,0,0,0,0,0,0,0


In [106]:
np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']])[:int(0.9*cryptic_mic_ordered.shape[0])]

array([0.12, 0.03, 0.25, ..., 0.25, 8.  , 0.06])

In [107]:
# np.savetxt('data_aa/RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']]), delimiter=',')   # X is an array
# np.savetxt('data_aa/train_RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']])[:int(0.9*cryptic_mic_ordered.shape[0])], delimiter=',')   # X is an array
# np.savetxt('data_aa/test_RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']])[int(0.9*cryptic_mic_ordered.shape[0]):], delimiter=',')   # X is an array

In [108]:
df.to_csv('data_aa/RIF_MIC_class.csv', index=False, header=True)

In [109]:
df.iloc[:int(0.9*cryptic_mic_ordered.shape[0]), :].to_csv('data_aa/train_RIF_MIC_class.csv', index=False, header=True)
df.iloc[int(0.9*cryptic_mic_ordered.shape[0]):, :].to_csv('data_aa/test_RIF_MIC_class.csv', index=False, header=True)

# fm Data

In [91]:
fm_aa = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_aa/cryptic_fm_transposed.csv')
fm_aa = fm_aa.loc[:, fm_aa.columns!='NA']
fm_aa = fm_aa.transpose()
fm_aa = fm_aa.iloc[1:,:]

In [113]:
fm_aa.to_csv('data_aa/fm_aa.csv', index=False)

In [108]:
mic_aa = cryptic_mic[cryptic_mic['wgs_id'].isin(fm_aa.index.unique())]#.iloc[:,14:27]
mic_aa['wgs_id'] = pd.Categorical(mic_aa['wgs_id'], categories=fm_aa.index.unique().tolist(), ordered=True)
mic_aa = mic_aa.sort_values('wgs_id')

/tmp/ipykernel_106569/1595918224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa['wgs_id'] = pd.Categorical(mic_aa['wgs_id'], categories=fm_aa.index.unique().tolist(), ordered=True)


In [112]:
mic_aa = mic_aa.iloc[:,14:27].astype('float')
mic_aa.to_csv('data_aa/mic_aa_fm.csv', index=False)

In [114]:
mic_aa.iloc[:int(mic_aa.shape[0]*0.9),:].to_csv('data_aa/mic_aa_fm_train.csv', index=False)
mic_aa.iloc[int(mic_aa.shape[0]*0.9):,:].to_csv('data_aa/mix_aa_fm_test.csv', index=False)

fm_aa.iloc[:int(fm_aa.shape[0]*0.9),:].to_csv('data_aa/fm_aa_train.csv', index=False)
fm_aa.iloc[int(fm_aa.shape[0]*0.9):,:].to_csv('data_aa/fm_aa_test.csv', index=False)

In [104]:
fm_aa.index.unique()

Index(['ERR4810491', 'ERR4810493', 'ERR4810494', 'ERR4810495', 'ERR4810497',
       'ERR4810498', 'ERR4810507', 'ERR4810509', 'ERR4810512', 'ERR4810516',
       ...
       'ERR8699755', 'ERR8686563', 'ERR8699762', 'ERR8699756', 'ERR8699805',
       'ERR8699231', 'ERR8699764', 'ERR8699789', 'ERR8699232', 'ERR8699267'],
      dtype='object', length=10824)

In [84]:
Nina_aa.transpose()

,0,1,2,3,4,5,6,7,8,9,...,9616,9617,9618,9619,9620,9621,9622,9623,9624,9625
Unnamed: 0,Chromosome_11_C,Chromosome_22_C,Chromosome_29_G,Chromosome_31_G,Chromosome_69_T,Chromosome_71_T,Chromosome_76_C,Chromosome_78_G,Chromosome_82_C,Chromosome_88_A,...,Chromosome_4338506_A,Chromosome_4338508_C,Chromosome_4338508_G,Chromosome_4338513_A,Chromosome_4338514_C,Chromosome_4338514_G,Chromosome_4338516_TCGCATTAACCGACGGGTCGGCCCAGG...,Chromosome_4338518_G,Chromosome_4338520_G,Chromosome_4338521_A
ERR4810489,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ERR4810491,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR4810493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR4810494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERR8699764,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ERR8699789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR8699232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ERR8699267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
